# Extract contours from GeoJSON

In [ ]:
jsonfile = "./Inputs/coastal_polygons.geojson"

In [ ]:
import json
import ogr
import osr
import datacube
import matplotlib.pyplot as plt

from dea_datahandling import load_ard
from dea_datahandling import mostcommon_crs
from dea_bandindices import calculate_indices
from dea_coastaltools import tidal_tag
from dea_spatialtools import contour_extract
from dea_plotting import display_map
from dea_plotting import rgb
from dea_plotting import map_shapefile
import xarray as xr

dc = datacube.Datacube(app='erosion_contours')

with open(jsonfile) as f:
    data = json.load(f)

In [ ]:
# for feature in data['features']:
#     print(feature['geometry']['type'])
#     print(feature['geometry']['coordinates'])

feature = data['features'][0]
print(feature)

In [ ]:
polygon = ogr.CreateGeometryFromJson(str(feature['geometry']))

source = osr.SpatialReference()
source.ImportFromEPSG(4326)

target = osr.SpatialReference()
target.ImportFromEPSG(3577)

transform = osr.CoordinateTransformation(source, target)
polygon.Transform(transform)

print(polygon)

In [ ]:
poly_envelope = polygon.GetEnvelope()
minX, maxX, minY, maxY = poly_envelope

In [ ]:
# Get dc.load params
x_range = (minX, maxX)
y_range = (minY, maxY)
time_range = ('2000', '2018')
crs = "EPSG:3577"
res = (-30, 30)
tide_range = (0.50, 1.00)

products = ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
measurements = ['nbart_red', 'nbart_green', 'nbart_blue', 'nbart_swir_1']

In [ ]:
query = {
    'y': y_range,
    'x': x_range,
    'time': time_range,
    'measurements': measurements,
    'crs': crs,
    'output_crs': crs,
    'resolution': res
}

landsat_ds = load_ard(
    dc=dc,
    products=products,
    group_by='solar_day',
    **query
)


In [ ]:
# Set the timesteps to visualise
timestep = 4

# Generate RGB plots at each timestep
rgb(landsat_ds, 
    index=timestep,
    percentile_stretch=(0.00, 0.95))


In [ ]:
landsat_ds = calculate_indices(landsat_ds, index='MNDWI', 
                               collection='ga_ls_3')

In [ ]:
# Calculate tides for each timestep in the satellite dataset
landsat_ds = tidal_tag(ds=landsat_ds, tidepost_lat=None, tidepost_lon=None)

# Print the output dataset with new `tide_height` variable
print(landsat_ds)

In [ ]:
# Calculate the min and max tide heights to include based on the % range
min_tide, max_tide = landsat_ds.tide_height.quantile(tide_range) 

# Plot the resulting tide heights for each Landsat image:
landsat_ds.tide_height.plot()
plt.axhline(min_tide, c='red', linestyle='--')
plt.axhline(max_tide, c='red', linestyle='--')
plt.show()


In [ ]:
# Keep timesteps larger than the min tide, and smaller than the max tide
landsat_filtered = landsat_ds.sel(time=(landsat_ds.tide_height > min_tide) &
                                       (landsat_ds.tide_height <= max_tide))
print(landsat_filtered)

In [ ]:
time_step = '1Y'

# Combine into summary images by `time_step`
landsat_summaries = (landsat_filtered.MNDWI
                     .compute()
                     .resample(time=time_step, closed='left')
                     .median('time'))

# Plot the output summary images
landsat_summaries.plot(col='time',
                       cmap='RdBu',
                       col_wrap=4,
                       vmin=-0.8,
                       vmax=0.8)
plt.show()


In [ ]:
# Set up attributes to assign to each waterline
attribute_data = {'time': [str(i)[0:10] for i in landsat_summaries.time.values]}
attribute_dtypes = {'time': 'str'}

# Extract waterline contours for the '0' water index threshold:
contour_gdf = contour_extract(
    z_values=[0],
    ds_array=landsat_summaries,
    ds_crs=landsat_ds.crs,
    ds_affine=landsat_ds.geobox.transform,
    output_shp=f'Outputs/contours.shp',
    attribute_data=attribute_data,
    attribute_dtypes=attribute_dtypes,
    min_vertices=50)

# Plot output shapefile over the first MNDWI layer in the time series
landsat_summaries.isel(time=0).plot(size=12, 
                                    cmap='Greys', 
                                    add_colorbar=False)
contour_gdf.to_crs(epsg=4326).to_file('Outputs/contours.geojson', driver='GeoJSON')

contour_gdf.plot(ax=plt.gca(), 
                 column='time', 
                 cmap='YlOrRd', 
                 legend=True, 
                 legend_kwds={'loc': 'lower right'})
plt.show()

print(contour_gdf)

## Compute change statistics

In [ ]:
import fiona
from shapely.geometry import mapping
from shapely.ops import nearest_points
from shapely.geometry import Point, LineString, MultiPoint
import geopandas as gpd

# Read in contours
# contours_gdf = gpd.read_file(f'output_data/{study_area}/{study_area}_{water_index}_{index_threshold}.shp')

# Set annual shoreline to use as a baseline
baseline_year = contour_gdf.index[0]
baseline_contour = contour_gdf.loc[baseline_year].geometry

# Set up output shapefile
schema = {'geometry': 'Point','properties': {'id': 'int'}}
baseline_points_shp = f'Outputs/test_statistics.shp' 

with fiona.open(baseline_points_shp, 'w', 'ESRI Shapefile', schema, crs=contour_gdf.crs) as output:
    
    # create points every 30 meters along the line
    for i, distance in enumerate(range(0, int(baseline_contour.length), 100)):
         point = baseline_contour.interpolate(distance)   
         output.write({'geometry': mapping(point), 'properties': {'id': i}})

In [ ]:
print(landsat_summaries)
print(baseline_year)

In [ ]:
# Read points in as geopandas
points_gdf = gpd.read_file(baseline_points_shp)

# Copy geometry to baseline point
points_gdf['p_baseline'] = points_gdf.geometry
baseline_x_vals = points_gdf.geometry.x
baseline_y_vals = points_gdf.geometry.y

# Get array of water index values for baseline time period 
baseline_array = landsat_summaries.isel(time = baseline_year)

print(baseline_array)

In [ ]:
# Iterate through all comparison years in contour gdf
for comp_year in contour_gdf.index.unique().values:

    #print(comp_year)

    # Set comparison contour
    comp_contour = contour_gdf.loc[comp_year].geometry
    
    # Find nearest point on comparison contour
    points_gdf[f'p_{comp_year}'] = points_gdf.apply(lambda x: 
                                                    nearest_points(x.p_baseline, comp_contour)[1], axis=1)
    
    # Compute distance between baseline and comparison year points
    points_gdf[f'{comp_year}'] = points_gdf.apply(lambda x: x.geometry.distance(x[f'p_{comp_year}']), axis=1)

    # Extract comparison array
    comp_array = landsat_summaries.isel(time = comp_year)
    
    # Convert baseline and comparison year points to geoseries to allow easy access to x and y coords
    comp_x_vals = gpd.GeoSeries(points_gdf[f'p_{comp_year}']).x
    comp_y_vals = gpd.GeoSeries(points_gdf[f'p_{comp_year}']).y
    
    # Sample NDWI values from arrays based on baseline and comparison points
    baseline_x_vals = xr.DataArray(baseline_x_vals, dims='z')
    baseline_y_vals = xr.DataArray(baseline_y_vals, dims='z')
    comp_x_vals = xr.DataArray(comp_x_vals, dims='z')
    comp_y_vals = xr.DataArray(comp_y_vals, dims='z')   
    points_gdf['index_comp_p1'] = comp_array.interp(x=baseline_x_vals, y=baseline_y_vals)
    points_gdf['index_baseline_p2'] = baseline_array.interp(x=comp_x_vals, y=comp_y_vals)
    
    # Compute directionality of change (negative = erosion, positive = accretion)    
    points_gdf['loss_gain'] = (points_gdf.index_baseline_p2 > points_gdf.index_comp_p1).astype(int).replace(to_replace=0, value=-1)
    points_gdf[f'{comp_year}'] = points_gdf[f'{comp_year}'] * points_gdf.loss_gain
    

# Get list of cols to keep
cols_to_keep = ['geometry'] + [str(val) for val in contour_gdf.index.unique().values]
    
# Keep required columns
points_gdf = points_gdf[cols_to_keep]
points_gdf = points_gdf.round(2)

print(points_gdf)


# print(points_gdf[['geometry']] + contour_gdf.index.unique().values.tolist())
# # print(points_gdf)

# for val in contour_gdf.index.unique().values.tolist():
#     print(val)
    


In [ ]:
from scipy import stats
import pandas as pd

def change_regress(row, x_vals, x_labels, std_dev=3):
    
    # Extract x (time) and y (distance) values
    x = x_vals
    y = row.values[1:].astype(np.float)
    
    # Drop NAN rows
    xy_df = np.vstack([x, y]).T
    is_valid = ~np.isnan(xy_df).any(axis=1)
    xy_df = xy_df[is_valid]
    valid_labels = x_labels[is_valid]
    
    # Remove outliers
    outlier_bool = (np.abs(stats.zscore(xy_df)) < float(std_dev)).all(axis=1)
    xy_df = xy_df[outlier_bool]
        
    # Compute linear regression
    lin_reg = stats.linregress(x=xy_df[:,0], 
                               y=xy_df[:,1])
    
    # Return slope, p-values and list of outlier years excluded from regression   
    return pd.Series({'slope': np.round(lin_reg.slope, 2), 
                      'pvalue': np.round(lin_reg.pvalue, 3),
                      'outliers': str(valid_labels[~outlier_bool]).replace('[', '').replace(']', '')})

In [ ]:
# Identify dates for regression
import numpy as np
x_years = np.array([int(i[:4]) for i in points_gdf.columns[1:]])

print(x_years)

In [ ]:
# Compute change rates
rate_out = points_gdf.apply(lambda x: change_regress(x, x_vals = x_years, x_labels = x_years, std_dev=3), axis=1)

points_gdf[['mov_rate', 'mov_sig', 'mov_outl']] = rate_out

print(points_gdf)

In [ ]:
# Set CRS
points_gdf.crs = str(landsat_ds.crs)

# Sort by descending absolute value and export
points_gdf.reindex(points_gdf.mov_rate.abs().sort_values().index).to_file(baseline_points_shp)

print(points_gdf)

In [ ]:
points_test = points_gdf[['geometry', 'mov_rate']]
points_test.crs = {'init' :'epsg:3577'}
points_new = points_test.to_crs({'init': 'epsg:4326'}) 

points_new.to_file('Outputs/contours_rateofchange.geojson', driver='GeoJSON')

print(points_new)

print(points_new.crs)

In [ ]:
type(points_gdf)